<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=dcd69a364c5ccc80dfaa90b71c32e59780ac9536194ce828750e044c9af480dd
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.2
    Uninstalling protobuf-6.33.2:
      Successfully uninstalled protobuf-6.33.2


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-12-17 14:20:58
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.28 C
CY Investment:  1.55 C
Reserve:  66.76 K
Current:  1.39 C
-------------------
Today PnL: -89.41 K (-0.64%)
Current PnL: -27.65 L (-17.86%)
CY Booked + Current PnL: -13.36 L (-8.63%)
-------------------
Total profit:  1.29 L
Total loss:  -28.94 L
-------------------
Total Booked + Current PnL: 13.62 L (10.66%)
Total Booked PnL: 41.27 L (32.29%)
Curr Year Booked PnL: 14.29 L (10.25%)
Prev Year Booked PnL: 26.98 L (21.11%)
Est FTT:  2.32 C
Est FTT PnL: 92.28 L (66.2%)
Deployed:  1.28 C
Current:  1.39 C
CAGR/XIRR %: 4.82%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TTKPRESTIG,770.00,-1.22,-19.08,23.69,0.09,19319.0,-19229.0,81548.0,89.33,31.0,M-SC,2.45,253.0,-1.00,0.59,7.14,OX40N,NTT,DURABLES
82,VOLTAS,1530.00,-0.82,8.18,10.64,19.69,22071.0,15693.0,207435.0,-2.32,55.0,X-MC,2.55,78.0,0.71,1.50,15.54,XY25,NTT,AC
50,MASFIN,398.61,-0.79,-3.55,26.54,22.05,25080.0,-3480.0,94500.0,-16.89,50.0,H-SC,6.86,164.0,-0.14,0.68,37.36,XR,ATH,FINANCE
39,INDIGOPNTS,1408.00,-1.30,-14.53,17.02,0.02,25393.0,-25362.0,149197.0,127.68,46.0,M-SC,14.37,234.0,-1.00,1.08,29.78,OX40N,NTT,PAINTS
43,ITC,452.00,-0.19,-1.73,12.73,10.78,30063.0,-4158.0,236160.0,-41.56,44.0,X-LC,1.99,5.0,-0.14,1.70,3.41,X40,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,3.19,-14.16,111.65,81.67,89905.0,-13286.0,80524.0,7238.89,53.0,L-SC,12.15,271.0,-0.15,0.58,55.94,XR,NTT,CERAMICS
13,BSOFT,831.70,1.38,-21.68,92.52,50.79,101920.0,-30493.0,110160.0,0.50,65.0,H-SC,6.44,151.0,-0.30,0.79,28.26,XR,ATH,IT
63,SFL,1287.00,1.11,-43.68,124.82,26.63,184349.0,-114526.0,147692.0,11.94,30.0,M-SC,11.87,239.0,-0.62,1.06,1.11,XY24,NTT,MISC
67,SONACOMS,806.63,0.96,-15.30,64.75,39.54,55478.0,-15479.0,85680.0,-33.70,50.0,M-SC,4.42,220.0,-0.28,0.62,20.82,AR,ATH,AUTO
16,CERA,9475.00,0.93,-29.36,75.38,23.89,93666.0,-51645.0,124258.0,-31.74,44.0,H-SC,10.77,157.0,-0.55,0.90,7.31,OX40N,NTT,CERAMICS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,COFFEEDAY,80.00,-4.20,-43.27,138.45,35.27,89184.0,-49133.0,64416.0,-55.73,42.0,L-SC,10.72,270.0,-0.55,0.46,57.66,XR,NTT,HOTELS
55,QUESS,424.00,-2.63,-31.24,107.74,42.84,48158.0,-20308.0,44698.0,-54.82,38.0,X-SC,35.42,83.0,-0.42,0.32,1.12,XY24,NTT,MISC
19,COLPAL,3726.84,-2.58,-20.09,77.08,41.51,162223.0,-52905.0,210460.0,-4.76,37.0,X-MC,7.35,61.0,-0.33,1.52,0.13,XY25,ATH,FMCG
71,TANLA,1943.92,-2.57,-37.34,250.60,119.69,425173.0,-101099.0,169662.0,-36.76,41.0,H-SC,7.97,156.0,-0.24,1.22,34.03,AR,ATH,IT
22,DIXON,18931.72,-2.45,-11.47,42.41,26.07,73294.0,-22399.0,172822.0,-56.97,37.0,X-MC,9.67,56.0,-0.31,1.25,7.64,X40N,ATH,IT


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-1.38,-2.62,113.51,107.91,163199.0,-3869.0,143775.0,-20.96,55.0,M-SC,12.56,216.0,-0.02,1.04,7.20,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,-1.30,-14.53,17.02,0.02,25393.0,-25362.0,149197.0,127.68,46.0,M-SC,14.37,234.0,-1.00,1.08,29.78,OX40N,NTT,PAINTS
77,TTKPRESTIG,770.00,-1.22,-19.08,23.69,0.09,19319.0,-19229.0,81548.0,89.33,31.0,M-SC,2.45,253.0,-1.00,0.59,7.14,OX40N,NTT,DURABLES
66,SIS,528.00,-0.29,-24.39,60.39,21.26,50695.0,-27086.0,83946.0,1980.31,49.0,H-SC,5.07,166.0,-0.53,0.61,13.42,OX40N,NTT,MISC
47,KANSAINER,340.00,-0.83,-25.69,52.70,13.47,105608.0,-69273.0,200394.0,-69.48,33.0,H-SC,5.66,158.0,-0.66,1.44,1.89,XY24,NTT,PAINTS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.0,0.5,6.85,61.22,72.27,110846.0,11613.0,181061.0,-8.35,65.0,M-LC,2.6,99.0,0.1,1.31,13.45,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,0.50,6.85,61.22,72.27,110846.0,11613.0,181061.0,-8.35,65.0,M-LC,2.60,99.0,0.10,1.31,13.45,XR,NTT,IT
25,FINCABLES,1641.55,-1.38,-2.62,113.51,107.91,163199.0,-3869.0,143775.0,-20.96,55.0,M-SC,12.56,216.0,-0.02,1.04,7.20,OX40N,ATH,CABLES
38,INDIAMART,4810.62,0.47,-4.10,115.55,106.72,136678.0,-5051.0,118285.0,-52.54,36.0,H-SC,3.29,139.0,-0.04,0.85,18.33,AR,ATH,MISC
85,ZYDUSLIFE,1286.17,0.34,-3.25,40.18,35.63,81472.0,-6806.0,202768.0,-16.74,38.0,H-MC,2.85,119.0,-0.08,1.46,12.88,AR,ATH,PHARMA
37,IEX,219.00,0.38,-4.35,55.39,48.63,105937.0,-8698.0,191256.0,-36.48,48.0,H-SC,14.07,136.0,-0.08,1.38,6.70,XR,NTT,MISC


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
9,BAJAJHFL,181.50,-0.72,-18.84,91.56,55.47,180793.0,-45827.0,197459.0,-28.03,23.0,X-MC,13.42,64.0,-0.25,1.42,1.17,X40N,ATH,FINANCE
53,PAGEIND,51605.08,-0.21,-9.93,42.24,28.11,76623.0,-20010.0,181400.0,-34.75,27.0,X-MC,13.19,55.0,-0.26,1.31,0.00,X40,ATH,APPARELS
8,AWL,485.00,-1.11,-23.92,101.75,53.50,233600.0,-72169.0,229582.0,-63.12,28.0,X-MC,3.29,58.0,-0.31,1.65,1.97,XY24,NTT,FMCG
11,BATAINDIA,2096.00,-1.00,-39.40,123.11,35.20,95994.0,-50696.0,77974.0,2.24,29.0,X-SC,16.29,92.0,-0.53,0.56,0.00,X40,NTT,FOOTWEAR
2,ABBOTINDIA,35195.00,-0.32,-7.03,26.99,18.07,44882.0,-12565.0,166290.0,-20.44,30.0,X-MC,6.02,63.0,-0.28,1.20,10.19,X40,ATH,PHARMA


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
15,CAMS,950.0,-2.15,-3.35,28.80,24.48,70102.0,-8448.0,243408.0,-81.54,38.0,X-SC,0.23,86.0,-0.12,1.75,19.01,X40N,NTT,MISC
43,ITC,452.0,-0.19,-1.73,12.73,10.78,30063.0,-4158.0,236160.0,-41.56,44.0,X-LC,1.99,5.0,-0.14,1.70,3.41,X40,NTT,FMCG
20,DABUR,735.0,-0.50,-3.28,48.57,43.70,116534.0,-8138.0,239930.0,-11.25,35.0,X-MC,2.26,72.0,-0.07,1.73,11.28,XY24,BTT,FMCG
33,HINDUNILVR,2922.0,-0.33,-11.56,28.52,13.67,64840.0,-29713.0,227350.0,-18.11,31.0,X-LC,2.31,14.0,-0.46,1.64,7.33,XY25,NTT,FMCG
36,ICICIPRULI,790.0,-1.45,4.04,25.66,30.73,52592.0,7951.0,204956.0,-20.34,52.0,X-MC,2.36,75.0,0.15,1.48,17.32,X40,ATH,INSURANCE


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ACC,3906.00,-0.64,-26.04,121.93,64.15,214597.0,-61951.0,176000.0,-56.75,31.0,X-SC,2.80,82.0,-0.29,1.27,0.00,XY24,BTT,CEMENT
11,BATAINDIA,2096.00,-1.00,-39.40,123.11,35.20,95994.0,-50696.0,77974.0,2.24,29.0,X-SC,16.29,92.0,-0.53,0.56,0.00,X40,NTT,FOOTWEAR
53,PAGEIND,51605.08,-0.21,-9.93,42.24,28.11,76623.0,-20010.0,181400.0,-34.75,27.0,X-MC,13.19,55.0,-0.26,1.31,0.00,X40,ATH,APPARELS
19,COLPAL,3726.84,-2.58,-20.09,77.08,41.51,162223.0,-52905.0,210460.0,-4.76,37.0,X-MC,7.35,61.0,-0.33,1.52,0.13,XY25,ATH,FMCG
31,HAVELLS,2069.16,-0.94,-10.45,47.93,32.48,140783.0,-34264.0,293727.0,-14.91,39.0,X-MC,6.10,67.0,-0.24,2.12,0.63,X40,ATH,ELECTRICAL


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,QUESS,424.00,-2.63,-31.24,107.74,42.84,48158.0,-20308.0,44698.0,-54.82,38.0,X-SC,35.42,83.0,-0.42,0.32,1.12,XY24,NTT,MISC
59,RELAXO,1176.00,-0.22,-48.18,193.93,52.31,145872.0,-69941.0,75219.0,-44.83,39.0,X-SC,7.10,91.0,-0.48,0.54,0.68,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-1.00,-39.40,123.11,35.20,95994.0,-50696.0,77974.0,2.24,29.0,X-SC,16.29,92.0,-0.53,0.56,0.00,X40,NTT,FOOTWEAR
51,MEDANTA,1486.00,0.37,-4.47,29.88,24.08,36237.0,-5675.0,121275.0,-12.23,34.0,X-SC,5.14,89.0,-0.16,0.87,14.27,XY24,NTT,HEALTHCARE
34,HONAUT,58357.33,-0.65,-17.01,72.71,43.32,98275.0,-27712.0,135160.0,-28.68,35.0,X-SC,4.88,90.0,-0.28,0.97,3.78,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.97,0.23,-11.67,36.65,20.71,122448.0,-44137.0,334100.0,-23.95,60.0,X-LC,4.25,1.0,-0.36,2.41,11.64,X40,ATH,IT
41,INFY,1972.00,0.46,8.94,23.23,34.24,79183.0,27976.0,340864.0,-15.29,61.0,X-LC,4.27,2.0,0.35,2.46,17.96,X40,NTT,IT
75,TMPV,600.00,0.49,-28.85,72.84,22.98,117076.0,-65168.0,160730.0,-26.74,32.0,X-LC,6.74,3.0,-0.56,1.16,1.09,XY24,NTT,AUTO
81,VBL,671.64,-1.47,-5.03,42.86,35.67,128561.0,-15886.0,299956.0,-16.39,49.0,X-LC,4.00,4.0,-0.12,2.16,8.13,X40N,ATH,FMCG
43,ITC,452.00,-0.19,-1.73,12.73,10.78,30063.0,-4158.0,236160.0,-41.56,44.0,X-LC,1.99,5.0,-0.14,1.70,3.41,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,3.19,-14.16,111.65,81.67,89905.0,-13286.0,80524.0,7238.89,53.0,L-SC,12.15,271.0,-0.15,0.58,55.94,XR,NTT,CERAMICS
50,MASFIN,398.61,-0.79,-3.55,26.54,22.05,25080.0,-3480.0,94500.0,-16.89,50.0,H-SC,6.86,164.0,-0.14,0.68,37.36,XR,ATH,FINANCE
13,BSOFT,831.70,1.38,-21.68,92.52,50.79,101920.0,-30493.0,110160.0,0.50,65.0,H-SC,6.44,151.0,-0.30,0.79,28.26,XR,ATH,IT
39,INDIGOPNTS,1408.00,-1.30,-14.53,17.02,0.02,25393.0,-25362.0,149197.0,127.68,46.0,M-SC,14.37,234.0,-1.00,1.08,29.78,OX40N,NTT,PAINTS
5,ASIANPAINT,3460.25,-0.30,-2.74,24.35,20.95,59626.0,-6899.0,244869.0,-3.72,47.0,X-LC,17.16,31.0,-0.12,1.77,31.92,X40,ATH,PAINTS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
13,BSOFT,831.70,1.38,-21.68,92.52,50.79,101920.0,-30493.0,110160.0,0.50,65.0,H-SC,6.44,151.0,-0.30,0.79,28.26,XR,ATH,IT
84,WIPRO,420.00,0.50,6.85,61.22,72.27,110846.0,11613.0,181061.0,-8.35,65.0,M-LC,2.60,99.0,0.10,1.31,13.45,XR,NTT,IT
41,INFY,1972.00,0.46,8.94,23.23,34.24,79183.0,27976.0,340864.0,-15.29,61.0,X-LC,4.27,2.0,0.35,2.46,17.96,X40,NTT,IT
27,GLAXO,3466.20,-0.13,3.66,34.23,39.15,70711.0,7292.0,206576.0,-16.74,53.0,X-MC,10.62,60.0,0.10,1.49,33.72,X40,ATH,PHARMA
73,TCS,4389.97,0.23,-11.67,36.65,20.71,122448.0,-44137.0,334100.0,-23.95,60.0,X-LC,4.25,1.0,-0.36,2.41,11.64,X40,ATH,IT


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.29
1,25,44.85
2,50,76.69


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.90
MC    30.94
LC    24.18
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.71
X40      23.08
X40N     12.98
XR        9.77
XY25      9.08
AR        9.04
OX40N     7.60
SR        0.99
MH        0.77
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    24.46
H-SC    23.79
X-LC    20.72
M-SC    11.69
X-SC     8.01
H-MC     4.82
L-SC     1.41
M-MC     1.32
M-LC     1.31
H-LC     1.13
L-LC     1.02
L-MC     0.34
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.36,-8.30,43.12
IT,13.05,-16.18,74.60
FINANCE,9.44,-21.65,74.56
MISC,7.27,-28.75,81.42
ELECTRICAL,6.01,-12.44,53.63
PAINTS,5.86,-13.68,30.56
INSURANCE,4.77,-2.44,37.50
PHARMA,4.15,-2.10,34.23
AUTO,2.83,-33.61,79.94


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3216453.0,21
AR,1317382.0,10
XR,1306566.0,13
X40,1023062.0,14
X40N,882222.0,9
OX40N,739749.0,10
XY25,381743.0,6
SR,283395.0,2
MH,77816.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3681343.0,25
M-SC,1438523.0,15
X-MC,1434492.0,16
X-LC,852637.0,11
X-SC,805756.0,8
H-MC,409807.0,3
L-SC,264639.0,3
M-LC,110846.0,1
H-LC,76188.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1290227.0      6
           AR         900814.0      5
M-SC       XY24       825523.0      6
H-SC       XR         786272.0      7
X-MC       X40        497157.0      7
           XY24       407739.0      3
           X40N       345302.0      4
H-SC       OX40N      342819.0      4
X-LC       X40        333390.0      5
X-SC       X40N       314249.0      3
M-SC       OX40N      311380.0      5
X-SC       XY24       298992.0      3
H-SC       SR         283395.0      2
X-LC       X40N       222671.0      2
H-MC       AR         213845.0      2
X-LC       XY24       198010.0      2
H-MC       XY24       195962.0      1
X-SC       X40        192515.0      2
X-MC       XY25       184294.0      2
L-SC       XR         179089.0      2
M-SC       XR         175085.0      2
           AR         126535.0      2
M-LC       XR         110846.0      1
X-LC       XY25        98566.0      2
L-SC       OX40N       85550.0      1
H-SC       MH          77816.0      1
H-LC       AR          76188.0      1
M-MC       XY25        61540.0      1
L-MC       XR          55274.0      1
L-LC       XY25        37343.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 24.0 seconds
